# Mal Webscraping

## Import Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

import datetime
import time

## Making fuctipon for more detailed information

<br> this info is found each anime's own webpage, thus it takes long

such as:
- anime type (TV/Movie/OVA)
- premaried (season and year)
- broadcast (day of week)
- stuio responsible
- duration
- age rating (G, PG13, R, R+)
- producers
- Source (Manga, Original, LN)
- Demograph (Shounen, Seinen)
- Themes (Gore, Military, Psycological)

In [381]:
# This function scrapes certain data from a spesified anime, though the use of the MAL refence sequence


def more_detail(mal_ref):
      genreArray = []
      themeArray = []

      url_Ref = f"https://myanimelist.net/anime/{mal_ref}"

      responseRef = requests.get(url_Ref).text
      soupRef = BeautifulSoup(responseRef, 'html.parser')

      time.sleep(2)

      typeAni      = (soupRef.select_one('span:-soup-contains("Type:")')).find_next().text

      if typeAni == 'TV':
            premiered  =   (soupRef.select_one('span:-soup-contains("Premiered:")').find_next_sibling().text)
            broadcast   = (soupRef.select_one('span:-soup-contains("Broadcast:")').find_next_sibling(string=True).split(' '*4)[1].split(' at ')[0])
     
            duration    = (soupRef.select_one('span:-soup-contains("Duration:")').find_next_sibling(string=True).split('  ')[1].split(' min. per')[0])

      
      elif typeAni == 'Movie':
 
            premiered   = np.nan
            broadcast   = np.nan
                 
            duration    = (soupRef.select_one('span:-soup-contains("Duration:")').find_next_sibling(string=True)).split('  ')[1].split('.\n')[0]


      elif typeAni == 'ONA' or typeAni == 'OVA':
            premiered   = np.nan
            broadcast   = np.nan

            duration    = (soupRef.select_one('span:-soup-contains("Duration:")').find_next_sibling(string=True)).split('  ')[1].split('.\n')[0].split('per epi')[0]
            ## TODO: Fine tuning

      elif typeAni == 'Special':
            premiered   = np.nan
            broadcast   = np.nan

            duration    = (soupRef.select_one('span:-soup-contains("Duration:")').find_next_sibling(string=True).split('  ')[1].split(' min. per')[0])

      else:                         ## For Music types, hard to find only Music type
            premiered   = np.nan
            broadcast   = np.nan

            duration    = (soupRef.select_one('span:-soup-contains("Duration:")').find_next_sibling(string=True)).split('  ')[1]#.split('.\n')[0]


      
      # TODO: Multiple Producers


      status      = (soupRef.select_one('span:-soup-contains("Status:")').find_next_sibling(string=True).split('  ')[1].split('\n')[0])
      producers   = (soupRef.select_one('span:-soup-contains("Producers:")').find_next_sibling().text)                              ## Not done
      studio      = (soupRef.select_one('span:-soup-contains("Studios:")').find_next_sibling().text)
      source      = (soupRef.select_one('span:-soup-contains("Source:")').find_next_sibling(string=True).split('  ')[1].split('\n')[0])
      ageRating    = (soupRef.select_one('span:-soup-contains("Rating:")').find_next_sibling(string=True).split(' ')[2])


      # Gets all the differnt genees
      #-------------------------

      # Genre try and exexption
      genre       = (soupRef.select_one('span:-soup-contains("Genre")'))          #Works both
      if genre != None:
            try:
                  genres = genre.find_next_siblings()
                  for i in genres:
                        genreArray.append(i.text)     
      
            ## THis is for single enties
            except AttributeError:
                  genre      =  genre.find_next_sibling()   # Gets one genre   

                  genreArray.append(genre.text)
      else:
            genreArray.append( 'None')


      #-------------------------------------------------
      # Demograph try and exexption
      demographic = (soupRef.select_one('span:-soup-contains("Demographic:")'))

      if demographic != None:
            try:
                  demographic = demographic.find_next_sibling().get_text()    

            except AttributeError:
                  demographic =  None
                  ## if no value found give none
      else:
            demographic = 'None'


      #-------------------------------------------------
      # Theme try and exexption
      theme      =  (soupRef.select_one('span:-soup-contains("Theme")'))

      if theme != None:
            try:            
                  themes = theme.find_next_siblings()
                  for i in themes:
                        themeArray.append(i.text)    

            except AttributeError:
                  theme      =  theme.find_next_sibling()
                  themeArray.append(theme.text)
      else:
            themeArray.append( 'None')

      
      #-------------------------------------------------
      # Only second enty eachtime
      
      genreArray = genreArray[::2]
      themeArray = themeArray[::2]
      #-------------------------


      return status, premiered, broadcast, producers,studio, source, duration, ageRating, demographic, themeArray, genreArray
      # return themeArray

In [299]:
# Small Tests

# okay = [49387,39486, 47917, 51535, 41467]

# print('status, premiered, broadcast, producers, studio, source, duration, age_rating, demograph, theme, genres')
# for i in okay:
#     print(i , ": " , more_detail(i))

status, premiered, broadcast, producers, studio, source, duration, age_rating, demograph, theme, genres
49387 :  ['Gore', 'Historical']
39486 :  ['Gag Humor', 'Historical', 'Parody', 'Samurai']
47917 :  ['CGDCT', 'Music']
51535 :  ['Gore', 'Military', 'Survival']
41467 :  ['None']


## Main webscraping function

Scrapes the data from the top anime main page

scrapes data like:
- 

In [378]:
titleListnew = []
ratingListnew = []
typeListnew = []
startDateListnew = []
endDateListnew = []
episodeListnew = []
membersListnew = []
refenceListnew = []

statusList = []
premieredList = []
broadcastList = []
producerList = []
studioList = []
sourceList = []
durationList = []
ageRatingList = []

demographList = []
genreList = []
themeList = []

## 50 entries takes apprx -- 3m45s
## 310 entries takes apprx -- 22m01s
## 500 entries takes apprx -- 37m10s

### Last score stuff at page: 13 750
for page in range(0,500,50):
    pages = f'https://myanimelist.net/topanime.php?limit={page}'

    newResponse = requests.get(pages).text
    soup = BeautifulSoup(newResponse, 'html.parser')

    time.sleep(10)

    
    commoon_link = soup.find_all('tr', class_ = 'ranking-list')
    
    for elemment in commoon_link:
        name = elemment.find('h3', attrs={'class':'hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3'})
        score = elemment.find(class_='score ac fs14').text.split('\n')[0]
        
        # # Have to deconstruct
        aniInfo = elemment.find(class_ = 'information di-ib mt4').text.split(' '*8)
        aniType = aniInfo[1].split(' ')[0]
        members = aniInfo[3].split(' ')[0].replace(',', '')
        
        epiTotal = aniInfo[1].split(' ')[1].split('(')[1]

        #TODO
        # if epiTotal == '?':
        #     print(0)
              
        
        # # consentrate
        airDate = aniInfo[2].split(' - ')[0]
        airEnd = aniInfo[2].split(' - ')[1].split('\n')[0]
        
        # ## URL Refense
        ref = elemment.find('div', class_= 'hoverinfo').get('id').split('info')[1]

        #-------------------------------

        status, premiered, broadcast, producers,studio, source, duration, ageRating, demographic, themeArray, genreArray = more_detail(ref)
        # status  = more_detail(ref)
        
        #-------------------------------

        # Appending to Lists
        titleListnew.append(name.text)          # correct
        ratingListnew.append(float(score))      # correct
        typeListnew.append(aniType)             # correct
        episodeListnew.append((epiTotal))       # correct
        startDateListnew.append(airDate)        # correct
        endDateListnew.append(airEnd)           # correct
        membersListnew.append(int(members))
        refenceListnew.append(int(ref))
        
        # ###--Exta info lists ---
        statusList.append(status)
        premieredList.append(premiered)
        broadcastList.append(broadcast)
        producerList.append(producers)
        studioList.append(studio)
        sourceList.append(source)
        durationList.append(duration)
        ageRatingList.append(ageRating)
        genreList.append(genreArray)

        demographList.append(demographic)
        themeList.append(themeArray)


        # print(epiTotal)
        # print(type(ref))


### Making Dataframe with the scraped data

In [379]:
multiPage = pd.DataFrame(data={'Rank': range(1,len(titleListnew)+1),
                               'Title':titleListnew,
                                'Rating': ratingListnew,
                                'Type': typeListnew,
                                'AirDate': startDateListnew,        #TODO: Convert to datetime here?
                                'EndDate': endDateListnew,
                                "TotalEpi": episodeListnew,
                                'Members' : membersListnew,
                                'MAL Ref': refenceListnew,
                                
                                'Status' : statusList,                            
                                'Premiered' : premieredList,                            
                                'Broadcast' : broadcastList,
                                'Studio' : studioList,                        
                                'Duration' : durationList,                        
                                'Age Ratingn' : ageRatingList,
                                'Genres' : genreList,
                                'Producer' : producerList,                            
                                'Demograph' : demographList,                            
                                'Themes' : themeList,                            
                                'Source' : sourceList                        
                                })

multiPage.set_index('Rank').reset_index()
# multiPage[['AirDate', 'EndDate']] = multiPage[['AirDate', 'EndDate']].apply(pd.to_datetime)
# multiPage['AirDate'] = (pd.to_datetime(multiPage['AirDate']))


multiPage.head()


,Rank,Title,Rating,Type,AirDate,EndDate,TotalEpi,Members,MAL Ref,Status,Premiered,Broadcast,Studio,Duration,Age Ratingn,Genres,Producer,Demograph,Themes,Source
0,1,Shingeki no Kyojin: The Final Season - Kankets...,9.21,Special,Mar 2023,2023,2,309208,51535,Currently Airing,NaN,NaN,MAPPA,1 hr. 1,R,"[Action, Drama, Suspense]",Production I.G,Shounen,"[Gore, Military, Survival]",Manga
1,2,Fullmetal Alchemist: Brotherhood,9.11,TV,Apr 2009,Jul 2010,64,3100193,5114,Finished Airing,Spring 2009,Sundays,Bones,24,R,"[Action, Adventure, Drama, Fantasy]",Aniplex,Shounen,[Military],Manga
2,3,Bleach: Sennen Kessen-hen,9.09,TV,Oct 2022,Dec 2022,13,397875,41467,Finished Airing,Fall 2022,Tuesdays,Pierrot,24,R,"[Action, Adventure, Fantasy]",TV Tokyo,Shounen,[None],Manga
3,4,Steins;Gate,9.08,TV,Apr 2011,Sep 2011,24,2387240,9253,Finished Airing,Spring 2011,Wednesdays,White Fox,24,PG-13,"[Drama, Sci-Fi, Suspense]",Frontier Works,None,"[Psychological, Time Travel]",Visual novel
4,5,Gintama°,9.07,TV,Apr 2015,Mar 2016,51,580202,28977,Finished Airing,Spring 2015,Wednesdays,Bandai Namco Pictures,24,PG-13,"[Action, Comedy, Sci-Fi]",TV Tokyo,Shounen,"[Gag Humor, Historical, Parody, Samurai]",Manga


### saving to csv for easier analysis

In [380]:
multiPage.to_excel('multiPages.xlsx', index=0)